In [1]:
from dotenv import dotenv_values
config = dotenv_values("../.env")

import os
import pandas as pd
from datetime import date, datetime, timedelta
import time

import random

from riotwatcher import LolWatcher, ApiError

""" 
API Rate Limits:
20 requests every 1 seconds(s)
100 requests every 2 minutes(s)
"""

""" initialize API """
watcher = LolWatcher(config['riotwatcher_api_key'])


""" user-defined variables """
list_summoner_name = ['AdAstraData', 'Storm and Sin']
var_summoner_region = 'euw1'

In [6]:
me_id = watcher.summoner.by_name(region=var_summoner_region, summoner_name='AdAstraData')['id']
me_puuid = watcher.summoner.by_name(region=var_summoner_region, summoner_name='AdAstraData')['puuid']
me_accountId = watcher.summoner.by_name(region=var_summoner_region, summoner_name='AdAstraData')['accountId']

watcher.summoner.by_name(region=var_summoner_region, summoner_name='AdAstraData')

{'id': 'SfjO89UvryyEXeh9iqgiwVkPoIo2TO8wE1EQA0flBZU0NSkZaYpDEyXo0A',
 'accountId': 'w5_T-v8uNiP_tXT6pw5pA_EhUNl8rKe8-4dFswyioNNqkfPI7Pl4NsQ7',
 'puuid': 'bZZNu22YOLpn8iDq1FCmW2u4fjW6hNGed3YKTh9LvQeNI65gSjd9EghgBq4HXyj9EKhH-bozh8EvjQ',
 'name': 'AdAstraData',
 'profileIconId': 5550,
 'revisionDate': 1670026996237,
 'summonerLevel': 30}

In [149]:
watcher.spectator.featured_games(region=var_summoner_region).keys()

tmp_df = pd.json_normalize(watcher.spectator.featured_games(region=var_summoner_region)['gameList'])

tmp_df['now'] = [
    datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S')
    for x in tmp_df['gameStartTime']
]

tmp_df.loc[0]['participants']
# var_encrypted_summoner_id = tmp_df.loc[0,'observers.encryptionKey']
# var_encrypted_summoner_id

[{'teamId': 100,
  'spell1Id': 32,
  'spell2Id': 4,
  'championId': 67,
  'profileIconId': 1109,
  'summonerName': 'Lucky Maddox',
  'bot': False},
 {'teamId': 100,
  'spell1Id': 4,
  'spell2Id': 3,
  'championId': 143,
  'profileIconId': 4540,
  'summonerName': 'Golem Golémique',
  'bot': False},
 {'teamId': 100,
  'spell1Id': 4,
  'spell2Id': 32,
  'championId': 64,
  'profileIconId': 577,
  'summonerName': 'Surely Not Pipou',
  'bot': False},
 {'teamId': 100,
  'spell1Id': 4,
  'spell2Id': 6,
  'championId': 81,
  'profileIconId': 906,
  'summonerName': 'Dønt tøuch my cs',
  'bot': False},
 {'teamId': 100,
  'spell1Id': 1,
  'spell2Id': 4,
  'championId': 41,
  'profileIconId': 5528,
  'summonerName': 'XpaJZ7810',
  'bot': False},
 {'teamId': 200,
  'spell1Id': 4,
  'spell2Id': 32,
  'championId': 711,
  'profileIconId': 847,
  'summonerName': 'jarjar fard',
  'bot': False},
 {'teamId': 200,
  'spell1Id': 6,
  'spell2Id': 4,
  'championId': 29,
  'profileIconId': 546,
  'summonerNam

In [150]:
watcher.summoner.by_name(region=var_summoner_region, summoner_name='Sunseî')

{'id': 'r6IQPxw9xlIYneVYN0Xq3SbPHjqUC2Xx9qwGPO35xth9_pk',
 'accountId': 'KF6VJ9TDlfzxkFUFYI4vSRYPKb0pfpYimHWfIogPQAAzZkk',
 'puuid': 'tXW_QF0tDR3bzq3D0MmlR56ruY5S9ZctPj9HFVQqJ84n9vpCngnCYdtwLOxLzPZGjtVHsv8txwcYMw',
 'name': 'Sunseî',
 'profileIconId': 1439,
 'revisionDate': 1669633584000,
 'summonerLevel': 432}

In [161]:
watcher.spectator.by_summoner(region=var_summoner_region, encrypted_summoner_id="r6IQPxw9xlIYneVYN0Xq3SbPHjqUC2Xx9qwGPO35xth9_pk").keys()

dict_keys(['gameId', 'mapId', 'gameMode', 'gameType', 'gameQueueConfigId', 'participants', 'observers', 'platformId', 'bannedChampions', 'gameStartTime', 'gameLength'])

In [156]:
pd.json_normalize(watcher.spectator.by_summoner(region=var_summoner_region, encrypted_summoner_id="r6IQPxw9xlIYneVYN0Xq3SbPHjqUC2Xx9qwGPO35xth9_pk"))

,gameId,mapId,gameMode,gameType,gameQueueConfigId,participants,platformId,bannedChampions,gameStartTime,gameLength,observers.encryptionKey
0,6174103697,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 32, 'spell2Id': 4...",EUW1,[],1670065176371,411,5K05WW1hopOgRcv/q4axpzJV0ddabNft


In [159]:
pd.json_normalize(watcher.spectator.by_summoner(region=var_summoner_region, encrypted_summoner_id="r6IQPxw9xlIYneVYN0Xq3SbPHjqUC2Xx9qwGPO35xth9_pk")['participants'])

,teamId,spell1Id,spell2Id,championId,profileIconId,summonerName,bot,summonerId,gameCustomizationObjects,perks.perkIds,perks.perkStyle,perks.perkSubStyle
0,100,32,4,67,1109,Lucky Maddox,False,5FhBXflSJFz9qXvBZitYIxy_2IQY090CgH5g-6A_rxXd4rk,[],"[8005, 9111, 9104, 8014, 8106, 8139, 5005, 500...",8000,8100
1,100,4,3,143,4540,Golem Golémique,False,ZcWy8JvJIYafqAJCuz2YzkyQnyDmoyXuS1fVqwrP1mPX_wQ,[],"[8351, 8304, 8345, 8347, 8009, 8017, 5007, 500...",8300,8000
2,100,4,32,64,577,Surely Not Pipou,False,ojZKTz3Pxbw0I1LhQ2UqeHgRE2jcqPWfaUf2PvTJiDGUBUA,[],"[8010, 9111, 9103, 8299, 8139, 8135, 5005, 500...",8000,8100
3,100,4,6,81,906,Dønt tøuch my cs,False,VAPYc84iQD1wApshNilxIJlGiGjqmDfUpgHY1z48RSTxy7Q,[],"[8008, 8009, 9103, 8299, 8226, 8210, 5005, 500...",8000,8200
4,100,1,4,41,5528,XpaJZ7810,False,TKopgCP_nlJ-fp3siYAlKrPyd7WTddwxutgaDcdLyDIvhX...,[],"[8369, 8304, 8321, 8347, 8135, 8138, 5008, 500...",8300,8100
5,200,4,32,711,847,jarjar fard,False,9Bl0zLGRw88QhYLZwmjt_C7W4be8i9MDxKNE6LGclxvYswk,[],"[8128, 8126, 8138, 8106, 8009, 8014, 5008, 500...",8100,8000
6,200,6,4,29,546,OkenekO,False,LB8_v8OZDhIQgEdoO6nMnkFImU_8XH0YR3nUnLxPya4PIUQ,[],"[8008, 8009, 9103, 8017, 8138, 8135, 5005, 500...",8000,8100
7,200,6,4,131,5173,Chîmera,False,6AA1HXTEAK_QMH86SqBCTNi-cj8mvxDppEzr3WHf9ND2vW8,[],"[8010, 8009, 9105, 8014, 8429, 8242, 5005, 500...",8000,8400
8,200,4,6,163,4025,Niko Senpai,False,r2UHwq4akEUMDlP1XzrRH5ppR30tP9GQD-FIHBOpAHDzzzo,[],"[8128, 8126, 8138, 8135, 8009, 8014, 5008, 500...",8100,8000
9,200,4,32,141,1439,Sunseî,False,r6IQPxw9xlIYneVYN0Xq3SbPHjqUC2Xx9qwGPO35xth9_pk,[],"[8128, 8143, 8138, 8106, 9111, 8014, 5008, 500...",8100,8000


In [158]:
tmp_df.loc[0]

gameId                                                            6174103697
mapId                                                                     12
gameMode                                                                ARAM
gameType                                                        MATCHED_GAME
gameQueueConfigId                                                        450
participants               [{'teamId': 100, 'spell1Id': 32, 'spell2Id': 4...
platformId                                                              EUW1
bannedChampions                                                           []
gameStartTime                                                  1670065176371
gameLength                                                               269
observers.encryptionKey                     5K05WW1hopOgRcv/q4axpzJV0ddabNft
now                                                      2022-12-03 10:59:36
Name: 0, dtype: object

In [65]:
# import requests

# url = "https://127.0.0.1:2999/liveclientdata/allgamedata"

# response = requests.request("GET",url, verify=False) ### avoid CERTIFICATE_VERIFY_FAILED
# liveclientdata = response.json()

d:\Program_Files\Python385\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [162]:
realTimestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S.%f')
elapsedMatchTimestamp = liveclientdata['gameData']['gameTime']



In [ ]:
gameMode = liveclientdata['gameData']['gameMode'] ### 1row only; with first {realTimestamp: elapsedMatchTimestamp}

# liveclientdata['activePlayer']['teamRelativeColors']
# liveclientdata['activePlayer']['summonerName']

In [95]:
pd.json_normalize(liveclientdata['events']['Events'])

,EventID,EventName,EventTime
0,0,GameStart,0.040636
1,1,MinionsSpawning,65.037918


In [123]:
# liveclientdata['activePlayer'].keys()

In [117]:
# pd.json_normalize(liveclientdata['activePlayer']['abilities'])

# liveclientdata['activePlayer']['abilities']['Q']['abilityLevel']
# liveclientdata['activePlayer']['abilities']['Q']['displayName']
# liveclientdata['activePlayer']['abilities']['Q']['id']

# liveclientdata['activePlayer']['abilities']['W']['abilityLevel']
# liveclientdata['activePlayer']['abilities']['W']['displayName']
# liveclientdata['activePlayer']['abilities']['W']['id']

# liveclientdata['activePlayer']['abilities']['E']['abilityLevel']
# liveclientdata['activePlayer']['abilities']['E']['displayName']
# liveclientdata['activePlayer']['abilities']['E']['id']

# liveclientdata['activePlayer']['abilities']['R']['abilityLevel']
liveclientdata['activePlayer']['abilities']['R']['displayName']
# liveclientdata['activePlayer']['abilities']['R']['id']


'Destiny'

In [86]:
pd.json_normalize(liveclientdata['activePlayer']['championStats']) ### add timestamp

,abilityHaste,abilityPower,armor,armorPenetrationFlat,armorPenetrationPercent,attackDamage,attackRange,attackSpeed,bonusArmorPenetrationPercent,bonusMagicPenetrationPercent,...,moveSpeed,omnivamp,physicalLethality,physicalVamp,resourceMax,resourceRegenRate,resourceType,resourceValue,spellVamp,tenacity
0,0.0,9.0,30.852749,0.0,1.0,59.474499,525.0,0.783109,1.0,1.0,...,330.0,0.0,0.0,0.0,421.335022,1.9624,MANA,421.335022,0.0,0.0


In [104]:
liveclientdata['activePlayer']['currentGold'] ### add timestamp

998.2861328125

In [105]:
liveclientdata['activePlayer']['level'] ### add timestamp

4

In [102]:
liveclientdata['activePlayer']['fullRunes'].keys()

dict_keys(['generalRunes', 'keystone', 'primaryRuneTree', 'secondaryRuneTree', 'statRunes'])

In [122]:
liveclientdata['activePlayer']['fullRunes']['generalRunes']
liveclientdata['activePlayer']['fullRunes']['keystone']
liveclientdata['activePlayer']['fullRunes']['primaryRuneTree']
liveclientdata['activePlayer']['fullRunes']['secondaryRuneTree']

liveclientdata['activePlayer']['fullRunes']['statRunes']

[{'id': 5005, 'rawDescription': 'perk_tooltip_StatModAttackSpeed'},
 {'id': 5008, 'rawDescription': 'perk_tooltip_StatModAdaptive'},
 {'id': 5003, 'rawDescription': 'perk_tooltip_StatModMagicResist'}]

In [130]:
liveclientdata['allPlayers'][0].keys()

dict_keys(['championName', 'isBot', 'isDead', 'items', 'level', 'position', 'rawChampionName', 'rawSkinName', 'respawnTimer', 'runes', 'scores', 'skinID', 'skinName', 'summonerName', 'summonerSpells', 'team'])

In [140]:
liveclientdata['allPlayers'][0]['championName']
liveclientdata['allPlayers'][0]['items']
liveclientdata['allPlayers'][0]['respawnTimer']
liveclientdata['allPlayers'][0]['runes']
liveclientdata['allPlayers'][0]['scores']
liveclientdata['allPlayers'][0]['summonerName']
liveclientdata['allPlayers'][0]['summonerSpells']
liveclientdata['allPlayers'][0]['team']


'ORDER'

In [ ]:
### per frame (per minute?) or is there a minor interval?

### describe summoner-match: champion, runes, purchased items, position (top, bottom, mid), match-up, basic stats, xy position, ...  
# gold
# cs
# kills
# deaths
# assists
# placed wards, kill wards
# number of times returned to base (recall)
# number of times that went to jungle camps
# number of times that went to a different position/ lane
# number of stand offs: # won, # lost
# number of healing potions


In [ ]:
from match_info.info_from_match import push_sql_tbl_summoner_matches

import pandas as pd 
import time

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.engine import URL

engine = create_engine("mssql+pyodbc://DESKTOP-CBU1M8B/DataScience?driver=ODBC+Driver+17+for+SQL+Server")

""" fetch all puuid from SQL tbl [DataScience].[dbo].[tbl_events_champion_level_up] """

list_tbl_events_champion_level_up_puuid_id = pd.read_sql_query("""
SELECT DISTINCT puuid_killer_id
FROM tbl_events_champion_kill
WHERE puuid_killer_id IS NOT NULL
""", con=engine)['puuid_killer_id'].tolist()

### this list will keep growing! 
print(len(list_tbl_events_champion_level_up_puuid_id)) 
### either limit by those that recently played in the last n days (e.g., with n = 3)
### AND/ OR focus search on those that played more than a minimum number of games recently
### by filtering directly on len(list(set(list_all_matches).difference(list_recorded_matches)))
### e.g. at least 5 games played in the last 3 days (need to add an additional rule to consider time)


28990


In [ ]:

# """ API GET puuid """

# list_summoner_name = ["Hålfhand", "twt bunnierina", "424PHL", "Halfhånd", "Stevie Nícks"]

# dict_summoner_puuid = {
#     "summoner_name": [],
#     "summoner_puuid": [],
# }

# for var_summoner_name in list_summoner_name:
#     dict_summoner_puuid["summoner_name"].append(var_summoner_name)
#     dict_summoner_puuid["summoner_puuid"].append(watcher.summoner.by_name(region=var_summoner_region, summoner_name=var_summoner_name)['puuid'])

# dict_summoner_puuid

{'summoner_name': ['Hålfhand',
  'twt bunnierina',
  '424PHL',
  'Halfhånd',
  'Stevie Nícks'],
 'summoner_puuid': ['e1YaYh4IlsmXcWiHZiMExvCwk6YVGqGV11RIsdv6DjqUucKNnHiPynl4EL1YvouzRH-Ovk4OobeT9w',
  'iyh9AVHGdU157WGpsGgMl6_WrEM-KrT4wHUg97BalxaRcMJmp0DgpUZhlMSgP4O5MNmsgvBoP4Q1bg',
  'IjED696BlMHhlXoRPaerwNP3JGLJ-DsKzRPJSNQpqYiCdyBduCeSmUpFG7YAWS9w-gUybVpgQgNYgA',
  '-ubDfM7ntX4mseW7cqJMyt7d7MhiqCvCvTatHruaJJG0wFfDeB1SjnIrTjGd5pVa5IOF7P5S1ZFskw',
  'U4Mfn3i6Lbnalf80hfqDPULhlFvk1AUB7S9m6lKZmNKpKiKMyF5CDiycdM6BE0AFS7iOlwgDcgyryw']}

In [ ]:

import pandas as pd 
import time

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.engine import URL

engine = create_engine("mssql+pyodbc://DESKTOP-CBU1M8B/DataScience?driver=ODBC+Driver+17+for+SQL+Server")

""" fetch all matches per puuid from SQL tbl [DataScience].[dbo].[tbl_summoner_matches] """

dict_matches_per_puuid = {}

rsample_list_tbl_events_champion_level_up_puuid_id = random.sample(list_tbl_events_champion_level_up_puuid_id, 100)

for var_summoner_puuid in rsample_list_tbl_events_champion_level_up_puuid_id:
# for var_summoner_puuid in [
#     'bZZNu22YOLpn8iDq1FCmW2u4fjW6hNGed3YKTh9LvQeNI65gSjd9EghgBq4HXyj9EKhH-bozh8EvjQ', ### AdAstraData
#     'pdmWhZjkPNzJ2FIcmd5YfkxyNmFTCVbMbL2WOaWgMdck2zc2QqNXaLRon8uEYrx30WwVO7uZ8lt0Gg', ### Storm and Sin

#     'e1YaYh4IlsmXcWiHZiMExvCwk6YVGqGV11RIsdv6DjqUucKNnHiPynl4EL1YvouzRH-Ovk4OobeT9w', ### Hålfhand
#     'iyh9AVHGdU157WGpsGgMl6_WrEM-KrT4wHUg97BalxaRcMJmp0DgpUZhlMSgP4O5MNmsgvBoP4Q1bg', ### twt bunnierina
#     'IjED696BlMHhlXoRPaerwNP3JGLJ-DsKzRPJSNQpqYiCdyBduCeSmUpFG7YAWS9w-gUybVpgQgNYgA', ### 424PHL
#     '-ubDfM7ntX4mseW7cqJMyt7d7MhiqCvCvTatHruaJJG0wFfDeB1SjnIrTjGd5pVa5IOF7P5S1ZFskw', ### Halfhånd
#     'U4Mfn3i6Lbnalf80hfqDPULhlFvk1AUB7S9m6lKZmNKpKiKMyF5CDiycdM6BE0AFS7iOlwgDcgyryw', ### Stevie Nícks

# #     # '-fewi6Tq1Pq_MUQTjNLRNEiQsb5bevVGwiOY9NCDfksQD5Y-Vy9-C3iFPMoe57RDfa3D25vMXEzZ0g', ### Neverpartccc
# #     # '-qxPpSEa53rVDAiVRKy6wgrxyw6LZ_QQmeEIN0n3qxILliYcbWVEQnh3DgstTXG3AQ9nK3Afod0FMQ', ### Paputô
# #     # '_0Xwns0hc-2623SHne0eZegGJ0QHUKvd0GEmSYp7RERw_uzuDn2pEKswglBaKoYHK1GnUnmi5xu0MA', ### eyesfull
# #     # '-caSnRkvodA1fIw2XI3ppLjKdlJLOOAMKZ87i3X2cxHL-BOVCX3q3DiQx93qBEY3ZD4NNnfKYZR-BA', ### Quennerrigneli
# #     # '-nLQhSOZUe8Pi5hGY-wB1TBOASebR5NYZqu4JiMU_-i15rkzRFTQsTi016stvNolddyftDWdx07X1Q', ### Emperor5
# #     # '-iqISG-IUzfIMVeHpJwbLrR2e5mUKgpS0Dqfzu7BF2_fyr5ohp_38wkKjRFAm59ySx1vrBF2iGBTIw', ### Epiphanyang
# ]:

    list_tbl_summoner_matches_match_id_per_puuid_id = pd.read_sql_query("""
    SELECT DISTINCT match_id
    FROM tbl_summoner_matches
    WHERE summoner_puuid = '%s'
    """ %(var_summoner_puuid), con=engine)['match_id'].tolist()

    dict_matches_per_puuid[var_summoner_puuid] = list_tbl_summoner_matches_match_id_per_puuid_id

""" for each puuid_id """
for var_summoner_puuid in rsample_list_tbl_events_champion_level_up_puuid_id:
# for var_summoner_puuid in [
#     'bZZNu22YOLpn8iDq1FCmW2u4fjW6hNGed3YKTh9LvQeNI65gSjd9EghgBq4HXyj9EKhH-bozh8EvjQ', ### AdAstraData
#     'pdmWhZjkPNzJ2FIcmd5YfkxyNmFTCVbMbL2WOaWgMdck2zc2QqNXaLRon8uEYrx30WwVO7uZ8lt0Gg', ### Storm and Sin

#     'e1YaYh4IlsmXcWiHZiMExvCwk6YVGqGV11RIsdv6DjqUucKNnHiPynl4EL1YvouzRH-Ovk4OobeT9w', ### Hålfhand
#     'iyh9AVHGdU157WGpsGgMl6_WrEM-KrT4wHUg97BalxaRcMJmp0DgpUZhlMSgP4O5MNmsgvBoP4Q1bg', ### twt bunnierina
#     'IjED696BlMHhlXoRPaerwNP3JGLJ-DsKzRPJSNQpqYiCdyBduCeSmUpFG7YAWS9w-gUybVpgQgNYgA', ### 424PHL
#     '-ubDfM7ntX4mseW7cqJMyt7d7MhiqCvCvTatHruaJJG0wFfDeB1SjnIrTjGd5pVa5IOF7P5S1ZFskw', ### Halfhånd
#     'U4Mfn3i6Lbnalf80hfqDPULhlFvk1AUB7S9m6lKZmNKpKiKMyF5CDiycdM6BE0AFS7iOlwgDcgyryw', ### Stevie Nícks

# #     # '-fewi6Tq1Pq_MUQTjNLRNEiQsb5bevVGwiOY9NCDfksQD5Y-Vy9-C3iFPMoe57RDfa3D25vMXEzZ0g', ### Neverpartccc
# #     # '-qxPpSEa53rVDAiVRKy6wgrxyw6LZ_QQmeEIN0n3qxILliYcbWVEQnh3DgstTXG3AQ9nK3Afod0FMQ', ### Paputô
# #     # '_0Xwns0hc-2623SHne0eZegGJ0QHUKvd0GEmSYp7RERw_uzuDn2pEKswglBaKoYHK1GnUnmi5xu0MA', ### eyesfull
# #     # '-caSnRkvodA1fIw2XI3ppLjKdlJLOOAMKZ87i3X2cxHL-BOVCX3q3DiQx93qBEY3ZD4NNnfKYZR-BA', ### Quennerrigneli
# #     # '-nLQhSOZUe8Pi5hGY-wB1TBOASebR5NYZqu4JiMU_-i15rkzRFTQsTi016stvNolddyftDWdx07X1Q', ### Emperor5
# #     # '-iqISG-IUzfIMVeHpJwbLrR2e5mUKgpS0Dqfzu7BF2_fyr5ohp_38wkKjRFAm59ySx1vrBF2iGBTIw', ### Epiphanyang
# ]:

    """ API GET puuid """
    var_summoner_name = watcher.summoner.by_puuid(region=var_summoner_region, encrypted_puuid=var_summoner_puuid)['name']

    print(var_summoner_name, var_summoner_puuid)

    """ fetch match_list for each week in the last n weeks (num_weeks) """

    list_all_matches = []

    num_weeks = 10 ### num_weeks = 10

    ### define current_week
    var_current_date = date.today()
    var_current_year, var_current_week, var_current_weekday = var_current_date.isocalendar()

    for week_num in range(num_weeks):
        
        # define iterator
        if var_current_week > week_num:
            var_week_iterator = var_current_week - week_num
            var_year_iterator = var_current_year
        else:
            var_week_iterator = 52 - (week_num - var_current_week) 
            var_year_iterator = var_current_year - 1
        
        # define startdate_iterator (Mon)
        var_startdate_iterator = datetime.fromisocalendar(var_year_iterator, var_week_iterator, 1)
        
        # define enddate_iterator (Sun)
        var_enddate_iterator = datetime.fromisocalendar(var_year_iterator, var_week_iterator, 7) + timedelta(days=1)

        # print(var_startdate_iterator, var_enddate_iterator)

        """ API GET week_matches played by puuid """
        list_week_matches = watcher.match.matchlist_by_puuid(
            region=var_summoner_region, 
            puuid=var_summoner_puuid,
            start_time=int(var_startdate_iterator.timestamp()), ### epoch timestamp in seconds
            end_time=int(var_enddate_iterator.timestamp()), ### epoch timestamp in seconds
            count=100, #count=100,
        )

        list_all_matches.extend(list_week_matches)

    """ API GET match_info for each match_id of all those found above """

    # print(list_all_matches)

    list_recorded_matches = dict_matches_per_puuid[var_summoner_puuid]

    print(var_summoner_name, list(set(list_all_matches).difference(list_recorded_matches)))

    # ### only record for those that have played at least 10 games in the current/last week:
    # if len(list(set(list_all_matches).difference(list_recorded_matches))) >= 10:
    
    """ for each match_id """
    for var_match_id in list(set(list_all_matches).difference(list_recorded_matches)):
        
        print(var_summoner_name, var_match_id) ###

        integration_datetime = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

        push_sql_tbl_summoner_matches(
            watcher, 
            var_summoner_puuid, 
            var_summoner_name, 
            var_summoner_region, 
            var_match_id, 
            engine, 
            integration_datetime
        )

        time.sleep(3)

    # else:
    #     pass


Bazypø 2DToXEFeM-9_Rk8kVrRxGNe5gjtOnw8_cGh4UsFyKFIi8X67pc-z8Ue8idsbyv-Kl7RYA2Yd4NADbg
Bazypø ['EUW1_6143727269', 'EUW1_6132757156', 'EUW1_6160841382', 'EUW1_6153100143', 'EUW1_6134392719', 'EUW1_6152817308', 'EUW1_6138588664', 'EUW1_6115367643', 'EUW1_6158130149', 'EUW1_6153125842', 'EUW1_6133006105', 'EUW1_6114280117', 'EUW1_6160762353', 'EUW1_6154183073', 'EUW1_6151733751', 'EUW1_6143498101', 'EUW1_6135411262', 'EUW1_6142237646', 'EUW1_6126401986', 'EUW1_6161765050', 'EUW1_6164358821', 'EUW1_6134993531', 'EUW1_6123940866', 'EUW1_6171348694', 'EUW1_6139985744', 'EUW1_6137122532', 'EUW1_6150131243', 'EUW1_6150446223', 'EUW1_6136049704', 'EUW1_6112928183', 'EUW1_6132931798', 'EUW1_6168617615', 'EUW1_6156769955', 'EUW1_6150262620', 'EUW1_6131157829', 'EUW1_6140800996', 'EUW1_6155010093', 'EUW1_6150344983', 'EUW1_6132810491', 'EUW1_6161796568', 'EUW1_6171319613', 'EUW1_6115399442', 'EUW1_6134665241', 'EUW1_6126602913', 'EUW1_6131053259', 'EUW1_6142267208', 'EUW1_6134726635', 'EUW1_6132444

KeyboardInterrupt: 

In [ ]:
var_match_id = "EUW1_6128022014"

json_res = watcher.match.timeline_by_match(region=var_summoner_region,match_id=var_match_id)

,frameInterval,frames,gameId,participants
0,60000,"[{'events': [{'realTimestamp': 1667043703528, ...",6128022014,"[{'participantId': 1, 'puuid': 'BWXg7uI8pQ8JIG..."


In [ ]:
json_res.keys()

dict_keys(['metadata', 'info'])

In [ ]:
json_res['info']['frames'][10]['participantFrames']['1'].keys()

dict_keys(['championStats', 'currentGold', 'damageStats', 'goldPerSecond', 'jungleMinionsKilled', 'level', 'minionsKilled', 'participantId', 'position', 'timeEnemySpentControlled', 'totalGold', 'xp'])

In [ ]:
json_res['info']['frames'][5]['participantFrames']['10']['position']

{'x': 13443, 'y': 2381}

In [ ]:
pd.json_normalize(json_res['info']['frames'][5]['events'])

,level,participantId,timestamp,type,levelUpType,skillSlot,itemId,creatorId,wardType
0,4.0,3.0,240136,LEVEL_UP,NaN,NaN,NaN,NaN,NaN
1,NaN,3.0,242349,SKILL_LEVEL_UP,NORMAL,3.0,NaN,NaN,NaN
2,NaN,8.0,244892,ITEM_PURCHASED,NaN,NaN,1052.0,NaN,NaN
3,4.0,2.0,244925,LEVEL_UP,NaN,NaN,NaN,NaN,NaN
4,NaN,8.0,245883,ITEM_PURCHASED,NaN,NaN,1052.0,NaN,NaN
5,NaN,8.0,247171,ITEM_PURCHASED,NaN,NaN,2055.0,NaN,NaN
6,NaN,8.0,247699,ITEM_DESTROYED,NaN,NaN,3340.0,NaN,NaN
7,NaN,8.0,247699,ITEM_PURCHASED,NaN,NaN,3364.0,NaN,NaN
8,NaN,2.0,252322,SKILL_LEVEL_UP,NORMAL,1.0,NaN,NaN,NaN
9,NaN,NaN,254040,WARD_PLACED,NaN,NaN,NaN,5.0,YELLOW_TRINKET


In [ ]:
from match_timeline.events_from_match_timeline import get_events_from_match_timeline

from match_timeline.champion_level_up import push_sql_tbl_events_champion_level_up
from match_timeline.skill_level_up import push_sql_tbl_events_skill_level_up

from match_timeline.ward_placed import push_sql_tbl_events_ward_placed
from match_timeline.ward_kill import push_sql_tbl_events_ward_kill

from match_timeline.champion_kill import push_sql_tbl_events_champion_kill

import pandas as pd 
import time

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.engine import URL

engine = create_engine("mssql+pyodbc://DESKTOP-CBU1M8B/DataScience?driver=ODBC+Driver+17+for+SQL+Server")

""" fetch all matches from SQL tbl [DataScience].[dbo].[tbl_summoner_matches] """

list_tbl_summoner_matches_match_id = pd.read_sql_query("""
SELECT DISTINCT match_id
FROM tbl_summoner_matches
""", con=engine)['match_id'].tolist()

list_tbl_summoner_matches_match_id = set(list_tbl_summoner_matches_match_id)

""" fetch all matches from SQL tbl [DataScience].[dbo].[tbl_events_champion_kill] """

list_tbl_events_champion_kill_match_id = pd.read_sql_query("""
SELECT DISTINCT match_id
FROM tbl_events_champion_kill
""", con=engine)['match_id'].tolist()

list_tbl_events_champion_kill_match_id = set(list_tbl_events_champion_kill_match_id)

list_recorded_matches = list_tbl_events_champion_kill_match_id

print(len(list(set(list_tbl_summoner_matches_match_id).difference(list_recorded_matches))))

""" for each match_id """
for var_match_id in list(set(list_tbl_summoner_matches_match_id).difference(list_recorded_matches)):

    watcher_match_timeline = watcher.match.timeline_by_match(region=var_summoner_region,match_id=var_match_id)

    print(datetime.now(), var_match_id)

    (
        # events_all,
        # events_pause_end,
        # events_ward_placed,
        # events_ward_kill,
        # events_champion_level_up,
        # events_skill_level_up,
        # events_item_purchases,
        # events_item_undo,
        # events_item_destroyed,
        # events_item_sold,
        events_champion_kill,
        # events_champion_special_kill,
        # events_elite_monster_kill,
        # events_turret_plate_destroyed,
        # events_building_kill,
    ) = get_events_from_match_timeline(watcher_match_timeline)

    integration_datetime = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

    df_puuid = pd.json_normalize(watcher_match_timeline['info']['participants'])
    dict_puuid = dict(df_puuid.values)

    if events_champion_kill.empty:
        print("pass") 
        ### create a pd.DataFrame with all columns with NULL value 
        # to register var_match_id on DB so that it skips 
        # the verification step at the level of 
        # list(set(list_tbl_summoner_matches_match_id).difference(list_recorded_matches))
        pass
    else:
        push_sql_tbl_events_champion_kill(var_match_id, events_champion_kill, dict_puuid, engine, integration_datetime)

    


62
2022-12-01 11:48:54.436427 EUW1_6167017121
pass
2022-12-01 11:48:54.618731 EUW1_6146158337
pass
2022-12-01 11:48:54.831489 EUW1_6144867653
pass
2022-12-01 11:48:55.020366 EUW1_6166028720
pass
2022-12-01 11:48:55.201927 EUW1_6164208429
pass
2022-12-01 11:48:55.385679 EUW1_6155348357
pass
2022-12-01 11:48:55.608074 EUW1_6166401718
pass
2022-12-01 11:48:55.855410 EUW1_6166819856
pass
2022-12-01 11:48:56.006366 EUW1_6161019112
pass
2022-12-01 11:48:56.526491 EUW1_6133428845
pass
2022-12-01 11:48:56.728551 EUW1_6164615010
pass
2022-12-01 11:48:56.870569 EUW1_6160362871
pass
2022-12-01 11:48:57.038406 EUW1_6141231886
pass
2022-12-01 11:48:57.195580 EUW1_6161475176
pass
2022-12-01 11:48:57.433268 EUW1_6161443102
pass
2022-12-01 11:48:57.617109 EUW1_6155133508
pass
2022-12-01 11:48:57.807519 EUW1_6143809771
pass
2022-12-01 11:48:57.977270 EUW1_6161474515
pass
2022-12-01 11:48:58.149261 EUW1_6162186068
pass
2022-12-01 11:48:58.308059 EUW1_6163886290
pass
2022-12-01 11:48:58.679517 EUW1_61623

In [ ]:
###

""" for each match_id """
for var_match_id in list_tbl_summoner_matches_match_id:

    watcher_match_timeline = watcher.match.timeline_by_match(region=var_summoner_region,match_id=var_match_id)

    (
        events_all,
        events_pause_end,
        events_ward_placed,
        events_ward_kill,
        events_champion_level_up,
        events_skill_level_up,
        events_item_purchases,
        events_item_undo,
        events_item_destroyed,
        events_item_sold,
        events_champion_kill,
        events_champion_special_kill,
        events_elite_monster_kill,
        events_turret_plate_destroyed,
        events_building_kill,
    ) = get_events_from_match_timeline(watcher_match_timeline)

    integration_datetime = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

    df_puuid = pd.json_normalize(watcher_match_timeline['info']['participants'])
    dict_puuid = dict(df_puuid.values)

    if events_ward_placed.empty:
        pass
    else:
        push_sql_tbl_events_ward_placed(var_match_id, events_ward_placed, dict_puuid, engine, integration_datetime)

    if events_ward_kill.empty:
        pass
    else:
        push_sql_tbl_events_ward_kill(var_match_id, events_ward_kill, dict_puuid, engine, integration_datetime)

    if events_champion_level_up.empty:
        pass
    else:
        push_sql_tbl_events_champion_level_up(var_match_id, events_champion_level_up, dict_puuid, engine, integration_datetime)

    if events_skill_level_up.empty:
        pass
    else:
        push_sql_tbl_events_skill_level_up(var_match_id, events_skill_level_up, dict_puuid, engine, integration_datetime)

    if events_champion_kill.empty:
        pass
    else:
        push_sql_tbl_events_champion_kill(var_match_id, events_champion_kill, dict_puuid, engine, integration_datetime)



In [ ]:
watcher_match_timeline

{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_6166464245',
  'participants': ['pzSt9aY2l5uIGLvcSxHye1ni88P2d6KLlcgBIC7FMHRHFnhjJB5ziPpxdKC3ZhOvfbMV7JE7Kssvcw',
   'kl-OG7WjpxgC8nzkBu54dnRQm2BNko1XAm0Id3tJ_eeeJ6AVAekbrWeVxe-o27r5rNtCJCv8-bwITg',
   'vof2EdjHrAFKyBokAn8HTLzRD6OJ8mJvQBybjGcxfDF0LNRSmVev1qHtW6t7iI5ajI2pZSIs8LHh2A',
   'sZ76RomH07SWUrj_VuyIS8EkdtVwNSIcmKkmJKy9aSEGDZpnhBVLSqFSZioeHz7K5H0VLj41zPaCGw',
   'SL0TEoksIwLmcRBGer0BpNEGtoO8CXZuYfjBiR24O3L7tOy5mP8tdPrKsKsznzZQ3Kiumdbf5s6kMQ',
   '3CNwFMMgPIZ_H6FQxwzdwnIcqruKTG0EczKogxgWw57sxOejmjMnT97NHXDmf4LhV6UArBfo6JYuAw',
   'u8bTReNaxa-2JKMCg2ShusJN2CLPpJHmb7GKQxKx77Jaz-CV2IFED4bRaytDp6vm31Kd-ahkTTgeiQ',
   'iMo7fomjD1qMYfqOA3d2TZkJ1E181_EPFEdnEQhlEr1yS__Idj8-doxosG-sa9Lstw91vwIyY-Weng',
   'M1jvBNS2vOckcErCYl3UnWjSkmoVXZXpFapHgHLvw1SIvzJk_OG9fCR_ov0rbxzAKeruDUkvt8tYoA',
   'pdmWhZjkPNzJ2FIcmd5YfkxyNmFTCVbMbL2WOaWgMdck2zc2QqNXaLRon8uEYrx30WwVO7uZ8lt0Gg']},
 'info': {'frameInterval': 60000,
  'frames': [{'events': [{'realT

In [ ]:
from match_timeline.events_from_match_timeline import get_events_from_match_timeline

""" for each match_id """
for var_match_id in ['EUW1_6167035432']: ### in list_tbl_summoner_matches:

    print(var_match_id)

    watcher_match_timeline = watcher.match.timeline_by_match(region=var_summoner_region,match_id=var_match_id)

    (
        # events_all,
        # events_pause_end,
        # events_ward_placed,
        # events_ward_kill,
        # events_champion_level_up,
        # events_skill_level_up,
        # events_item_purchases,
        # events_item_undo,
        # events_item_destroyed,
        # events_item_sold,
        events_champion_kill,
        # events_champion_special_kill,
        # events_elite_monster_kill,
        # events_turret_plate_destroyed,
        # events_building_kill,
    ) = get_events_from_match_timeline(watcher_match_timeline)

    integration_datetime = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

    df_puuid = pd.json_normalize(watcher_match_timeline['info']['participants'])
    dict_puuid = dict(df_puuid.values)

import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.engine import URL

engine = create_engine("mssql+pyodbc://DESKTOP-CBU1M8B/DataScience?driver=ODBC+Driver+17+for+SQL+Server")

# dt = events_ward_placed

watcher_match_timeline
# events_champion_kill.type.value_counts()
# dt.dropna(axis=1, inplace=True)

# dt['match_id'] = match_id
# dt['puuid_id'] = dt['creatorId'].map(dict_puuid)
# dt['event_id'] = dt['type'].str.lower()
# dt['event_timestamp'] = dt['timestamp']
# dt['ward_type'] = dt['wardType'].str.lower()
# dt['integration_datetime'] = integration_datetime

# dt = dt[['integration_datetime', 'match_id', 'puuid_id', 'event_id', 'event_timestamp', 'ward_type']]

# df_tmp = pd.read_sql_query("SELECT * from tbl_events_ward_placed WHERE match_id = '%s'" %(match_id), con=engine)

# if df_tmp.empty:
#     dt.to_sql('tbl_events_ward_placed', con=engine, if_exists='append', index=False)
# else:
#     pass

EUW1_6167035432


{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_6167035432',
  'participants': ['3JvNMQ83mLFtWHKy7XOtUFHmNH0eD1MkC8GCZ-2xBKAUe-Y2TPj5FIi4awdXxxcEy1Qj_HpiJljhrQ',
   'f--v0JdPOYg-55PA9yJJ31iDVHJ7DHzBq74QzVE3mNpi4mBuvxI_CGQg2ysrPkycK0rShHXacTYVyQ',
   'RMXFI8xLJbeOhUbIns_egmo9FVjHO4HlNfwooXLeq6v9fB8QyCFWOWm5eT7tyTgIdY8IwGP8hu71Cg',
   'EGMlizlnhGmv3b9cmZycJ_lQSmbbpfY99kkAk9N94DM7joHzHZJVsqhjbQ2Qja6HfCxIu74T4X0kiA',
   'Jp86BnXEaPsCxlXd_SXlBhNA2oeanYQD1zD3htbCOyRQmmGLWPNX3Y1hsRBjPxAeC1w_p76KjIsjkQ',
   '8kpJyoGxMSYU9WZ7L4MOuqJv2moOPu4rB9j3ETB7Bbv1Hp3c_VVCfOT_j4zE5U5Mp5BlMsyFeVGl6Q',
   'txybaOKZHmwE6PhtuAdTzkov7JuiduBkrN-aIvn0PUsCz800EFVPbwf_HBZN2dHEd3xmFvkb259fZg',
   'bZZNu22YOLpn8iDq1FCmW2u4fjW6hNGed3YKTh9LvQeNI65gSjd9EghgBq4HXyj9EKhH-bozh8EvjQ',
   'iAZP--AXoAP_MhMXP-Y_jXUdBI8Y86w3OhVbMsoxQXuAEfZ2KAa0bxWRooEJcvy5VmvRmhPp_ZuWPA',
   'ToPm7JRep-BLvP-jJBFt72Cpflf94ilVb-Cbo59YZ5rtipCbawicnZOO9sB2G9gT3Thx2NvOAxb6jQ']},
 'info': {'frameInterval': 60000,
  'frames': [{'events': [{'realT

In [ ]:

# """ fetch item_list from data_dragon and create dict with item_id : item_name """
# data_dragon_versions = watcher.data_dragon.versions_for_region(region=var_region)
# data_dragon_item_version = data_dragon_versions['n']['item']

# item_list = watcher.data_dragon.items(data_dragon_item_version)

# dict_item_id_item_name = {}

# for item_id in item_list['data'].keys():
#     df_item_id = pd.json_normalize(data=item_list['data'][item_id])
#     item_name = df_item_id['name'].to_string(index=False)
#     dict_item_id_item_name[item_id] = item_name


In [ ]:

watcher_match['info'].keys()
# dict_keys(['gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 
# 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 
# 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])


# """ global information per participant/ summoner """
watcher_match['info']['participants'][2]

# """ global information per team """
# watcher_match['info']['teams'][0]


NameError: name 'watcher_match' is not defined

In [ ]:
# watcher_match_timeline.keys()
# # dict_keys(['metadata', 'info'])

# watcher_match_timeline['metadata'].keys()
# # dict_keys(['dataVersion', 'matchId', 'participants'])

# watcher_match_timeline['info'].keys()
# # dict_keys(['frameInterval', 'frames', 'gameId', 'participants'])

# """ game_participants """
# watcher_match_timeline['info']['participants']


In [ ]:

""" 
Champion Kill
"""


def push_sql_tbl_events_champion_kill(engine, events_champion_kill, dict_puuid, match_id = watcher_match_timeline['metadata']['matchId']):


push_sql_tbl_events_champion_kill(engine, events_champion_kill, dict_puuid)


In [ ]:

# dtt = pd.json_normalize(events_champion_kill['victimDamageReceived'])

# # for this match_id

# # for each champion kill:
# for i in range(dtt.shape[0]):

# # for each contribution to champion kill:
#     for j in range(len(dtt.iloc[i])):

#         if dtt.iloc[i, j] == 'None':
#             pass
#         else:
#             pass
#             # print(dtt.iloc[i, j])
 
#         print(dtt.iloc[i, j])

In [ ]:


events_champion_special_kill

events_elite_monster_kill

events_turret_plate_destroyed

events_building_kill


In [ ]:

# for var_match_id in df_match_info['match_id'].unique():
#     df_tmp = pd.read_sql_query("SELECT * from tbl_match_info WHERE match_id = '%s'" %(var_match_id), con=engine)
    
#     if df_tmp.empty:
#         df_append = df_match_info.query("match_id == @var_match_id")
#         df_append.to_sql('tbl_match_info', con=engine, if_exists='append', index=False)
#     else:
#         pass

In [ ]:
### 1 FRAME PER MIN
datetime.fromtimestamp(watcher_match_timeline['info']['frames'][1]['timestamp'] / 1000).strftime('%H:%M:%S')
datetime.fromtimestamp(watcher_match_timeline['info']['frames'][2]['timestamp'] / 1000).strftime('%H:%M:%S')
datetime.fromtimestamp(watcher_match_timeline['info']['frames'][3]['timestamp'] / 1000).strftime('%H:%M:%S')

### INSIDE EACH FRAME, per participant
watcher_match_timeline['info']['frames'][1]['participantFrames']['1'].keys()
# dict_keys(['championStats', 'currentGold', 'damageStats', 'goldPerSecond', 
# 'jungleMinionsKilled', 'level', 'minionsKilled', 'participantId', 'position', 
# 'timeEnemySpentControlled', 'totalGold', 'xp'])

watcher_match_timeline['info']['frames'][20]['participantFrames']['1']['totalGold']

watcher_match_timeline['info']['frames'][20]['participantFrames']['1']['xp']


10209

In [ ]:
dict_match_info = {
    "match_id": [],
    "team_id": [],
    "participant_puuid": [],
    "summoner_name": [],
    "gold_earned": [],
}

""" for each match_id """
for var_match_id in last_matches_puuid[0:100]:

    """ GET summoner puuids """
    watcher_match = watcher.match.by_id(region=var_region, match_id=var_match_id)

    for puuid_num in range(len(watcher_match['metadata']['participants'])):
        puuid_features = watcher_match['info']['participants'][puuid_num]

        dict_match_info['match_id'].append(var_match_id)
        dict_match_info['team_id'].append(puuid_features['teamId'])
        dict_match_info['participant_puuid'].append(puuid_features['puuid'])
        dict_match_info['summoner_name'].append(puuid_features['summonerName'])
        dict_match_info['gold_earned'].append(puuid_features['goldEarned'])
    
    """ time delay with time.sleep """
    time.sleep(1)


In [ ]:
df_match_info = pd.DataFrame.from_dict(dict_match_info)
df_match_info['team_id'] = ['Blue' if team_id == 100 else 'Red' for team_id in df_match_info['team_id']]
df_match_info

list_all_match_participants = df_match_info['summoner_name'].unique()
len(list_all_match_participants)

853

In [ ]:
len(list_all_match_participants)

853

In [ ]:
list_all_match_participants[25:26]

array(['Alecostry22'], dtype=object)

In [ ]:
""" user-defined variables """
var_region = "euw1"
# var_summoner_name= "AdAstraData"

dict_match_info = {
    "match_id": [],
    "team_id": [],
    "participant_puuid": [],
    "summoner_name": [],
    "gold_earned": [],
}

for var_participant_name in list_all_match_participants[27:30]:
    
    """ GET participant puuid """
    summoner_puuid = watcher.summoner.by_name(region=var_region, summoner_name=var_participant_name)
    var_participant_puuid = summoner_puuid['puuid']
    
    """ GET last 100 matches played by puuid """
    last_matches_puuid = watcher.match.matchlist_by_puuid(
        region=var_region, 
        puuid=var_participant_puuid, 
        count=100
    )

    """ time delay with time.sleep """
    time.sleep(1)

    """ for each match_id """
    for var_match_id in last_matches_puuid[0:100]:
        
        """ GET summoner puuids """
        watcher_match = watcher.match.by_id(region=var_region, match_id=var_match_id)
        
        for puuid_num in range(len(watcher_match['metadata']['participants'])):
            puuid_features = watcher_match['info']['participants'][puuid_num]
            
            dict_match_info['match_id'].append(var_match_id)
            dict_match_info['team_id'].append(puuid_features['teamId'])
            dict_match_info['participant_puuid'].append(puuid_features['puuid'])
            dict_match_info['summoner_name'].append(puuid_features['summonerName'])
            dict_match_info['gold_earned'].append(puuid_features['goldEarned'])

        """ time delay with time.sleep """
        time.sleep(1)


# EUW1_5021326873


In [ ]:
df_match_info = pd.DataFrame.from_dict(dict_match_info)
df_match_info['team_id'] = ['Blue' if team_id == 100 else 'Red' for team_id in df_match_info['team_id']]
df_match_info

,match_id,team_id,participant_puuid,summoner_name,gold_earned
0,EUW1_6161945478,Blue,A2zEx9varFrcDbkKlqdGuD0dr5dCO0pcB4MRqH6N6Qha6J...,poutmasmurf,6898
1,EUW1_6161945478,Blue,MEVanfuJg9PZqtZbaJPB2JbkU9gbb83q9wDJobu06PXpnf...,FriiZoK7,9587
2,EUW1_6161945478,Blue,h3C486P2TXijAzf80wqo8wuscyhoo1Qypzd1pvu0JEYb6a...,haus300,8852
3,EUW1_6161945478,Blue,me53WpNeg3rqK2j0xb30wxMWFQ37MMZdF1RzPvSi38SR1j...,godbutbackwards,6209
4,EUW1_6161945478,Blue,Jl4DjLQgMwQK-9xga9-ddMeEgsq5NGUFy_Gy9O-SQkHQhW...,davidjimi,5610
...,...,...,...,...,...
1926,EUW1_5162267977,Blue,-C2u2z2GvgyjMrwuVLWcrkVN3joW392KZSMWtLRCHAO5v_...,popcgae,4749
1927,EUW1_5162267977,Blue,n9xTZEVS8IP4C0arauuyyHyrI_NvR0bz7D-uQV5rJdY9Nc...,Kanriee,7994
1928,EUW1_5162267977,Blue,joR0O9lnLuhRuU8z0OfKJxJU9xxOEguG-HCI_Yarrjm-Fp...,SyRuB1,14513
1929,EUW1_5162267977,Blue,GQIS690djRh5tNELDpJNdxcvyVlwGcqShzRI3krMOeJbjj...,skeith241,6349


In [ ]:
import sqlite3
from sqlalchemy import create_engine
engine = create_engine('sqlite:///riotwatcher_api_dt_match.db', echo=False)

# df_append = pd.DataFrame(
#     [['test', 'test', 'test', 'test', 'test']],
#     columns = ['match_id', 'team_id', 'participant_puuid', 'summoner_name', 'gold_earned']
# )

# df_append.to_sql('tbl_match_info', con=engine, if_exists='append', index=False)

for var_match_id in df_match_info['match_id'].unique():
    df_tmp = pd.read_sql_query("SELECT * from tbl_match_info WHERE match_id = '%s'" %(var_match_id), con=engine)
    
    if df_tmp.empty:
        df_append = df_match_info.query("match_id == @var_match_id")
        df_append.to_sql('tbl_match_info', con=engine, if_exists='append', index=False)
    else:
        pass


In [ ]:
df = pd.read_sql_query("SELECT DISTINCT * from tbl_match_info", con=engine)
df

,match_id,team_id,participant_puuid,summoner_name,gold_earned
0,test,test,test,test,test
1,EUW1_6162024635,Blue,pdmWhZjkPNzJ2FIcmd5YfkxyNmFTCVbMbL2WOaWgMdck2z...,Storm and Sin,6124
2,EUW1_6162024635,Blue,bAbkdqqlpnuDlKFo32I_PBfi-yO2R8eBcD4qo9hqkAZ8Nm...,Ariiial,4539
3,EUW1_6162024635,Blue,bZZNu22YOLpn8iDq1FCmW2u4fjW6hNGed3YKTh9LvQeNI6...,AdAstraData,6727
4,EUW1_6162024635,Blue,9fZyZmC705BV1nflyxYDE_qtc4EnxO0SsCjIZ6DG2PV-Fi...,Glockenhorst,9177
...,...,...,...,...,...
20413,EUW1_5162267977,Blue,-C2u2z2GvgyjMrwuVLWcrkVN3joW392KZSMWtLRCHAO5v_...,popcgae,4749
20414,EUW1_5162267977,Blue,n9xTZEVS8IP4C0arauuyyHyrI_NvR0bz7D-uQV5rJdY9Nc...,Kanriee,7994
20415,EUW1_5162267977,Blue,joR0O9lnLuhRuU8z0OfKJxJU9xxOEguG-HCI_Yarrjm-Fp...,SyRuB1,14513
20416,EUW1_5162267977,Blue,GQIS690djRh5tNELDpJNdxcvyVlwGcqShzRI3krMOeJbjj...,skeith241,6349


In [ ]:
df = pd.read_sql_query("SELECT DISTINCT * from tbl_match_info WHERE summoner_name = 'AdAstraData' AND gold_earned > 0", con=engine)
df['gold_earned'].astype(int).describe()
### AdAstraData

count      112.000000
mean      9048.589286
std       3601.078609
min        550.000000
25%       6694.000000
50%       8803.500000
75%      11406.500000
max      18333.000000
Name: gold_earned, dtype: float64

In [ ]:
df = pd.read_sql_query("SELECT DISTINCT * from tbl_match_info WHERE summoner_name = 'Storm and Sin' AND gold_earned > 0", con=engine)
df['gold_earned'].astype(int).describe()
### Storm and Sin

count      100.000000
mean     11145.490000
std       3752.877472
min        807.000000
25%       9047.500000
50%      11338.000000
75%      13439.500000
max      19373.000000
Name: gold_earned, dtype: float64

In [ ]:
### ACTION_ITEMS : 
# 4. Add new data and test if append respects existing match info

In [ ]:
list_match_item_purchased = []

for var_match_id in last_matches_puuid[0:10]:

    """ store participant's puuids """
    dict_participants_puuids = {}

    match_participants = watcher.match.by_id(region=var_region, match_id=var_match_id)
    list_match_participants = match_participants['metadata']['participants']
    
    for idx in range(len(list_match_participants)):
        participant_id = idx + 1
        dict_participants_puuids[str(participant_id)] = list_match_participants[idx]

    """ fetch match_timeline """
    match_info = watcher.match.timeline_by_match(region=var_region,match_id=var_match_id)
    match_timeline = pd.json_normalize(data=match_info['info']['frames'], record_path='events')

    """ add match_id to match_timeline"""
    match_timeline['match_id'] = var_match_id

    """ convert match_timeline' timestamp to elapsed minutes"""
    match_timeline['timestamp_timedelta'] = pd.to_timedelta(match_timeline['timestamp'], unit='ms')
    match_timeline['elapsed_minutes'] = round(match_timeline['timestamp_timedelta']/ pd.Timedelta(minutes=1),3)
    
    """ subset for item_purchased """
    item_purchased = match_timeline.query("type == 'ITEM_PURCHASED'").dropna(axis=1)

    """ retrieve item_name """
    item_purchased['item_id'] = item_purchased['itemId'].astype(int)
    item_purchased['item_name'] = [dict_item_id_item_name[str(item)] for item in item_purchased['item_id']]

    """ retrieve summoner_puuid """
    item_purchased['participant_id'] = item_purchased['participantId'].astype(int)
    item_purchased['summoner_puuid'] = [dict_participants_puuids[str(id)] for id in item_purchased['participant_id']]

    """ subset for @my_puuid """
    item_purchased = item_purchased.query("summoner_puuid == @my_puuid")

    """ select cols - elapsed_minutes, item_name, summoner_puuid """
    item_purchased = item_purchased[
        [
            'match_id', 
            'elapsed_minutes',
            'item_name',
            'summoner_puuid'
        ]
    ]
    
    """ create dict with item_purchased per timestamp per participant (summoner) """
    dict_item_purchased = item_purchased.to_dict(orient='records')
    list_match_item_purchased.append(dict_item_purchased)

# record end time
end_timer = time.time()

# number of API calls
num_api_calls_per_cycle = 1
num_api_cycles = len(last_matches_puuid)
num_api_calls = num_api_cycles * num_api_calls_per_cycle

print("Execution time:",(end_timer-start_timer), "s")
print("Number of API calls:",num_api_calls)
print("API calls per second:",round(num_api_calls/(end_timer-start_timer),2))

Execution time: 14.254740476608276 s
Number of API calls: 100
API calls per second: 7.02


In [ ]:
""" obtain avg. elapsed_minutes per item_name (last matches)"""
from itertools import chain

df_match_item_purchased = pd.DataFrame(list(chain.from_iterable(list_match_item_purchased)))

avg_min_Dorans_Ring = df_match_item_purchased.query('item_name == "Doran\'s Ring"')['elapsed_minutes'].describe()
avg_min_Dorans_Ring

count    24.000000
mean      0.259708
std       0.146596
min       0.097000
25%       0.155500
50%       0.209000
75%       0.322000
max       0.614000
Name: elapsed_minutes, dtype: float64

In [ ]:
### ACTION_ITEMS : 
# 1. need to test first for API call before proceeding (hitting the API's rate limits as it is) 
# 2. need to add order_num to item_name to distinguish between similar item_names bought at different timestamps
# 3. .describe() to be applied per item_name_order_num 